# I. 금통위 의사록

## 1. PDF 다운로드

In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
import re

for i in range(1,21): # 마지막 페이지 입력하기
    URL = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex='+str(i)
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    list_minutes = soup.select('main[id="content"] div[class="bdLine type2"] > ul > li')
    for minutes in list_minutes:
        title = minutes.select('a')[0].text.strip()
        pdf_check = minutes.select('a')[2].text.strip()
        pdf_check2 = minutes.select('a')[4].text.strip()
        sub_link = minutes.select('a')[2].attrs['href']
        sub_link2 = minutes.select('a')[4].attrs['href']
        if re.findall(r'(?<=차\)).*',title)[0][-2] != '.':  # (1111.11.11)형식이면
            pdf_date = datetime.strptime(re.findall(r'(?<=차\)).*',title)[0][1:-1],"%Y.%m.%d")
            pdf_date2 = pdf_date.strftime('%Y.%m.%d')
        else:
            pdf_date = datetime.strptime(re.findall(r'(?<=차\)).*',title)[0][1:-2],"%Y.%m.%d")
            pdf_date2 = pdf_date.strftime('%Y.%m.%d')
        if 'pdf' in pdf_check:
            link = 'https://www.bok.or.kr' + sub_link
        else:
            link = 'https://www.bok.or.kr' + sub_link2
        file_res = requests.get(link)
        with open(f"{pdf_date2}.pdf", 'wb') as f:
            f.write(file_res.content)

f.close()

##      2. PDF파일에서 전체 텍스트 추출

In [ ]:
from tika import parser
from datetime import datetime, timedelta, date
f = open('mp_report.csv','w',newline='')
wr = csv.writer(f)
start_date = date(2011,1,1)
end_date = date(2020,12,31)
ds = start_date
while True:
    try:
        filename = f'./금융통화위원회 의사록/{ds.strftime("%Y.%m.%d")}.pdf'
        parsed = parser.from_file(filename, requestOptions={'timeout':120})
#        print(ds)
#        print(parsed['content'].strip())
        wr.writerow([ds,parsed['content'].strip()])
    except:
        pass
    ds += timedelta(1)
    if ds == end_date:
        break
f.close()
pd.read_csv('mp_report.csv',header=None)

## 3. 추출한 텍스트 정제하기

### a) 불필요 str 제거해주는 함수 및 의사록 섹션 2&3 분리하는 함수 불러오기

In [ ]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [ ]:
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

### b) 의사록 데이터프레임 읽어오기

In [ ]:
import os
from tika import parser
import pandas as pd
import csv
mp_report = pd.read_csv("mp_report.csv", header=None, encoding='utf-8')
mp_report.columns = ['date','내용']
mp_report['content']=None
mp_report

### c) 의사록 섹션 2,3 추출

In [ ]:
from ekonlpy.tag import Mecab

mecab = Mecab()
content = []

for i in range(200):
    minutes = preprocess_minutes(mp_report['내용'][i])
    sections = minutes[1][1]+minutes[1][2]
    
    if sections == []:
        content.append([])
    else:
        content.append(sections)

mp_report['content'] = content
mp_report.drop('내용', axis=1, inplace=True)


mp_report.to_csv(r'./mp_report_df.csv', index=False, encoding='utf-8') # 편의상 덮어쓰기

##### 여기서 나온 결과는 콘텐트 번갈아가며 빈 리스트([])만 들어가있는 경우가 있음 코드 정리하며 합치기

### d) 처리 결과 확인

In [ ]:
import pandas as pd
mp_report_df = pd.read_csv("mp_report_df.csv", encoding='utf8')
mp_report_df

# II. 채권분석리포트

In [ ]:
# option 1
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import datetime
!pip install openpyxl
import openpyxl
wb = openpyxl.Workbook()
sheet = wb.active
sheet.append(["날짜","제목","본문"])
for i in range(1,101): # 마지막 페이지 입력하기 101
    URL = 'https://finance.naver.com/research/debenture_list.nhn?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2011-01-01&writeToDate=2021-12-31&x=0&y=0&page='+str(i)
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    list_minutes = soup.select('div[id="contentarea_left"] div[class="box_type_m"] tr')
    for minutes in list_minutes:
        try:
            title=minutes.select('a')[0].text
            sub_link = minutes.select('a')[1].attrs['href']
            date = datetime.strptime('20'+ minutes.select('.date')[0].text, "%Y.%m.%d")
            if str(sub_link).endswith('.pdf'):
                print(date.strftime('%Y.%m.%d'))
                print(title)
                print(sub_link)
                file_res = requests.get(sub_link)
                sheet.append([ds.strftime('%Y.%m.%d'),title,sub_link])
            else:
                pass
        except:
            pass
wb.save("bond_report.csv")

In [ ]:
# option 2 - x download pdf/ getting urls only
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import csv
f = open('bond_report.csv','w',newline='')
wr = csv.writer(f)
for i in range(1,102): # 마지막 페이지+1 입력
    URL = 'https://finance.naver.com/research/debenture_list.nhn?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2011-01-01&writeToDate=2021-12-31&x=0&y=0&page='+str(i)
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    list_minutes = soup.select('div[id="contentarea_left"] div[class="box_type_m"] tr')
    for minutes in list_minutes:
        try:
            title=minutes.select('a')[0].text
            sub_link = minutes.select('a')[1].attrs['href']
            date = datetime.strptime('20'+ minutes.select('.date')[0].text, "%Y.%m.%d")
            date_convert = date.strftime('%Y.%m.%d')
            print(date_convert)
            print(title)
            print(sub_link)
            wr.writerow([date_convert,title,sub_link])
        except:
            pass
f.close()

In [ ]:
#링크안 pdf text 긁어오기
import os
from tika import parser
import pandas as pd
import csv
bond_report = pd.read_csv("bond_report.csv", header=0)
bond_report.columns = ['date','title','url']
content = []
for url in bond_report['url']:
    try:
        content.append(parser.from_file(url)['content'].replace('\n', ''))
    except:
        content.append("NA")
        pass
bond_report['content'] = content
bond_report 
bond_report.to_csv(r'/Users/soojinkim/kdigital/mini_project_1/bond_report_df.csv')

# III. 네이버 뉴스 기사
네이버뉴스 사이트에서 "금리"키워드 검색 후 언론3사(이데일리, 연합인포맥스, 연합뉴스)선택

In [ ]:
import openpyxl
wb = openpyxl.Workbook()
sheet = wb.active
sheet.append(["날짜","기사명","본문"])
#은이님 수정
from bs4 import BeautifulSoup
import requests
import csv
import re
import datetime
start_date = datetime.datetime(2013,1,1)
end_date = datetime.datetime(2014,1,1)
total_days = int((end_date-start_date).days)
ds=start_date #시작일자
errorcount=0
for day in range(1,total_days+1):
    page = 1
    while True:
        headers = {'user-agent':'Mozilla/5.0','cookie':'news_office_checked=1001,1018,2227'}
        url = f"https://search.naver.com/search.naver?&where=news&query=%EA%B8%88%EB%A6%AC&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={ds.strftime('%Y.%m.%d')}&de={ds.strftime('%Y.%m.%d')}&docid=&nso=so:r,p:from{ds.strftime('%Y%m%d')}to{ds.strftime('%Y%m%d')},a:all&mynews=1&start={page}&refresh_start=0"
        response = requests.get(url,headers=headers)
        soup = BeautifulSoup(response.text,"html.parser")        
        li_list = soup.select(".list_news > li")
        for li in li_list:
            try:
#                print(ds.strftime('%Y.%m.%d'))
                if li.select('div .news_area a[href]')[0]['href'] == 'http://www.einfomax.com':
                    title = li.select('div .news_area a[title]')[0].text
                    article_url = li.select('div .news_area a[title]')[0]['href']
                    article_response = requests.get(article_url, headers=headers)
                    article_soup = BeautifulSoup(article_response.text,"html.parser")
                    article = article_soup.find(id = 'article-view-content-div')
                    body = re.sub(r'[\n\t]','',article.text.strip())
                else:
                    title = li.select('div .news_area a[title]')[0].text
                    article_url = li.select('div .news_area a[href]')[1]['href']
                    article_response = requests.get(article_url, headers=headers)
                    article_soup = BeautifulSoup(article_response.text,"html.parser")
                    article = article_soup.find(id = 'articleBodyContents')
                    body = article.text.strip()
#                 print(title)
#                 print(body)
#                 print("") 
                sheet.append([ds.strftime('%Y.%m.%d'),title,body])
            except:
                errorcount+=1
        button=soup.select('.sc_page > a')[1]['aria-disabled']
        page+=10
        if button == 'true':
            ds+=datetime.timedelta(1)
            break
wb.save("2020.csv")

# IV. 콜금리

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
f = open("call.csv", "w")
f.write("날짜,콜금리,\n")
for i in range(1,400): 
    url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page='+str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    list_call = soup.select('div[class="section_exchange2"] tbody > tr')
    for call in list_call:
        date=str(call.select('td')[0].text.strip())
        call_index=str(call.select('td')[1].text.strip())
        f.write(str(date)+","+str(call_index)+'\n')
f.close()

# V. 기준금리

In [1]:
# 노가다 복붙 10년안에바뀐횟수 <20회